In [5]:
#   Converting Camera Matrix (openCV) to openGl
#   Sources:
#   http://www.info.hiroshima-cu.ac.jp/~miyazaki/knowledge/teche0092.html
#   https://fruty.io/2019/08/29/augmented-reality-with-opencv-and-opengl-the-tricky-projection-matrix/


import cv2
import numpy as np
import math
np.set_printoptions(suppress=True,
                    formatter={'float_kind': '{:f},'.format})

In [3]:
def compare_projections(point, camera_mtx, opengl_mtx):
    print(point)
    #     
    # screen_point, _ = cv2.projectPoints(np.array([point]), np.zeros(3), np.zeros(3), camera_mtx, np.zeros(5))
    # print(screen_point)

    #Note: we obtain the same result with this: (that's what cv2.projectPoints basically does: multiply points with camera matrix and then divide result by z coord)
    open_cv = camera_mtx.dot(point)/point[2]
    print(open_cv)


    #### OpenGL projection
    #we flip the point z coord, because in opengl camera is oriented along -Oz axis
    point[2] = -point[2]
    point2 = np.hstack([point,1.0]) #we add vertex w coord (usually done in vertex shader before multiplying by projection matrix)
    #we get the point in clip space

    clip_point = opengl_mtx.dot(point2)

    #NOTE: what follows "simulates" what happens in OpenGL after the vertex shader.
    #This is necessary so that we can make sure our projection matrix will yield the correct result when used in OpenGL
    #we get the point in NDC

    ndc_point = clip_point / clip_point[3]
    #we get the screen coordinates
    viewport_point = (ndc_point + 1.0)/2.0 * np.array([w, h, 1.0, 1.0])
    #opencv Oy convention is opposite of OpenGL so we reverse y coord
    viewport_point[1] = h - viewport_point[1]
    
    print(viewport_point)
    print()
    
    

In [17]:
# Camera Matrix (OpenCV)
camera_mtx = np.loadtxt(fname='camera_matrix.txt', delimiter=',')
info =  np.loadtxt(fname='image_info.txt', delimiter=',')

cx = camera_mtx[0][2]
cy = camera_mtx[1][2]

fx = camera_mtx[0][0]
fy = camera_mtx[1][1]

w = info[0]
h = info[1]

near = 1
far = 20

opengl_mtx = np.array([
    [2*fx/w, 0.0, (w - 2*cx)/w, 0.0],
    [0.0, -2*fy/h, (h - 2*cy)/h, 0.0],
    [0.0, 0.0, (-far - near) / (far - near), -2.0*far*near/(far-near)],
    [0.0, 0.0, -1.0, 0.0]
])

print('OpenCV Matrix')
print(camera_mtx, '\n')
print('OpenGL Matrix')
print(opengl_mtx, '\n')
print('Image Properties')
print('width : ',w)
print('height: ',h,'\n')

points = [
    np.array([1.0, 2.0, 15.0]),
    np.array([20.0, 40.0, 100.0]),
    np.array([300.0, 600.0, 1000.0])
]

for point in points:
   compare_projections(point=point,camera_mtx=camera_mtx,opengl_mtx=opengl_mtx)



OpenCV Matrix
[[3549.288187, 0.000000, 1724.224632,]
 [0.000000, 3551.909378, 2289.571598,]
 [0.000000, 0.000000, 1.000000,]] 

OpenGL Matrix
[[2.044521, 0.000000, 0.006783, 0.000000,]
 [0.000000, -1.536293, 0.009701, 0.000000,]
 [0.000000, 0.000000, -1.105263, -2.105263,]
 [0.000000, 0.000000, -1.000000, 0.000000,]] 

Image Properties
width :  3472.0
height:  4624.0 

[1.000000, 2.000000, 15.000000,]
[1960.843845, 2763.159515, 1.000000,]
[1960.843845, 2808.016319, 0.982456, 1.000000,]

[20.000000, 40.000000, 100.000000,]
[2434.082269, 3710.335350, 1.000000,]
[2434.082269, 3755.192153, 1.042105, 1.000000,]

[300.000000, 600.000000, 1000.000000,]
[2789.011088, 4420.717225, 1.000000,]
[2789.011088, 4465.574029, 1.051579, 1.000000,]

